In [1]:
import pandas as pd
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.master("spark://outline.ru-central1.internal:7077").getOrCreate()

23/04/15 21:41:12 WARN Utils: Your hostname, outline resolves to a loopback address: 127.0.1.1; using 10.129.0.21 instead (on interface eth0)
23/04/15 21:41:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/15 21:41:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
_columns=["Id","Name","CountsOfReview"]
dfs = [spark.read
       .option("header", True)
       .option("quote", "\"")
       .option("escape", "\"")
       .option("multiLine", True)
       #.schema(customSchema)
       .option("inferSchema", True) 
       .csv('data/'+file)
       #.select(_columns)
       for file in os.listdir(r'data') if file.startswith("book")]

In [5]:
len(dfs)

23

In [7]:
merged_df = dfs[0]
for i in range(1, len(dfs)):
    merged_df = merged_df.unionByName(dfs[i], allowMissingColumns=True)    

In [8]:
merged_df

DataFrame[Id: int, Name: string, Authors: string, ISBN: string, Rating: double, PublishYear: int, PublishMonth: int, PublishDay: int, Publisher: string, RatingDist5: string, RatingDist4: string, RatingDist3: string, RatingDist2: string, RatingDist1: string, RatingDistTotal: string, CountsOfReview: int, Language: string, pagesNumber: int, Description: string, Count of text reviews: int]

In [9]:
merged_df.repartition(1).write.parquet('data/parquet')

In [10]:
merged_df.repartition(1).write.csv('data/merged')

In [13]:
%time
df_csv=spark.read.csv("data/merged/part-00000-26cb797a-491c-4cba-a52b-4ae54e03765f-c000.csv")

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [16]:
%time
df=spark.read.parquet("data/parquet/part-00000-e8c84c05-9b92-4623-bbf1-4424f9b5e8b9-c000.snappy.parquet")

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [4]:
df.show(5)

+-------+--------------------+----------------+----------+------+-----------+------------+----------+--------------------+-----------+-----------+-----------+-----------+-----------+---------------+--------------+--------+-----------+--------------------+---------------------+
|     Id|                Name|         Authors|      ISBN|Rating|PublishYear|PublishMonth|PublishDay|           Publisher|RatingDist5|RatingDist4|RatingDist3|RatingDist2|RatingDist1|RatingDistTotal|CountsOfReview|Language|pagesNumber|         Description|Count of text reviews|
+-------+--------------------+----------------+----------+------+-----------+------------+----------+--------------------+-----------+-----------+-----------+-----------+-----------+---------------+--------------+--------+-----------+--------------------+---------------------+
|1700000|Montmorency On Th...|  Eleanor Updale|0439606764|  3.89|       2005|           1|         4|       Orchard Books|      5:346|      4:501|      3:313|       2

In [17]:
df.select("Id","Name","CountsOfReview") \
  .orderBy(df.CountsOfReview.desc()) \
  .show(10) 

+-------+--------------------+--------------+
|     Id|                Name|CountsOfReview|
+-------+--------------------+--------------+
|2767052|The Hunger Games ...|        154447|
|  41865|Twilight (Twiligh...|         94850|
|  19063|      The Book Thief|         87685|
|4667024|            The Help|         76040|
|      3|Harry Potter and ...|         75911|
|   3636|The Giver (The Gi...|         57034|
|  43641| Water for Elephants|         52918|
|2429135|The Girl with the...|         52225|
| 136251|Harry Potter and ...|         52088|
|  28187|The Lightning Thi...|         48630|
+-------+--------------------+--------------+
only showing top 10 rows



In [18]:
from pyspark.sql.functions import desc

top_publishers = df.groupBy("Publisher") \
                  .mean("pagesNumber") \
                  .orderBy(desc("avg(pagesNumber)")) \
                  .limit(10)

top_publishers.show()

+--------------------+------------------+
|           Publisher|  avg(pagesNumber)|
+--------------------+------------------+
|Crafty Secrets Pu...|         1807321.6|
|    Sacred-texts.com|          500000.0|
|Department of Rus...| 322128.5714285714|
|Logos Research Sy...|          100000.0|
|Encyclopedia Brit...|           32642.0|
|Progressive Manag...|        19106.3625|
|Still Waters Revi...|10080.142857142857|
|P. Shalom Publica...|            8539.0|
|Hendrickson Publi...|            6448.0|
|            IEEE/EMB|            6000.0|
+--------------------+------------------+



In [19]:
top_years = df.groupBy("PublishYear") \
              .count() \
              .orderBy(desc("count")) \
              .limit(10)

top_years.show()

+-----------+------+
|PublishYear| count|
+-----------+------+
|       2007|129507|
|       2006|122374|
|       2005|117639|
|       2004|105733|
|       2003|104345|
|       2002| 95537|
|       2001| 88228|
|       2000| 87290|
|       2008| 80265|
|       1999| 80155|
+-----------+------+



In [20]:
from pyspark.sql.functions import split

df2 = df.alias('df2')
id(df2) == id(df)  # False
cols_to_update = ["RatingDist1", "RatingDist2", "RatingDist3", "RatingDist4", "RatingDist5", "RatingDistTotal"]

for col in cols_to_update:
    df2 = df2.withColumn(col, split(df2[col], ":").getItem(1).cast("integer"))


In [21]:
from pyspark.sql.functions import col

filtered_df = df2.filter(col("RatingDistTotal") > 500)

std_df = filtered_df.withColumn("RatingStd", (((5 - col("Rating"))**2 * col("RatingDist5") + (4 - col("Rating"))**2 * col("RatingDist4") + (3 - col("Rating"))**2 * col("RatingDist3") + (2 - col("Rating"))**2 * col("RatingDist2") + (1 - col("Rating"))**2 * col("RatingDist1"))/col("RatingDistTotal"))**0.5)
top10_df = std_df.sort(col("RatingStd").desc()).limit(10)
top10_df.select("Name", "RatingStd").show()


+--------------------+------------------+
|                Name|         RatingStd|
+--------------------+------------------+
|Scientology: The ...|1.6768965920829089|
|Scientology: The ...|1.6754156681545784|
|Para Entrenar a u...|1.6599055615998013|
| To Train Up a Child|1.6595374499170727|
|Para Entrenar a u...|1.6594896798570395|
|The Bluebook: A U...|1.5883197973323213|
|The Bluebook: A U...|1.5883197973323213|
|Dianetics: The Mo...|1.5621133225369277|
|Dianetics: The Mo...|1.5608985177269006|
|Dianetica: La Cie...| 1.560359470866146|
+--------------------+------------------+



In [33]:
from pyspark.sql.functions import avg, count

filtered_df = df2.filter((col("PublishYear") >= 1000) & 
                         (col("PublishYear") <= 2022) & 
                         (col("RatingDistTotal") > 500))
grouped_df = filtered_df.groupBy("PublishYear")\
                        .agg(avg("Rating").alias("AverageRating"), count("*").alias("BookCount"))
filtered_grouped_df = grouped_df.filter(col("BookCount") > 1000)

print("top-10 best year average rating")
filtered_grouped_df.orderBy(filtered_grouped_df.AverageRating.desc()) \
                   .show(10) 

print("top-10 worst year average rating")
filtered_grouped_df.orderBy(filtered_grouped_df.AverageRating) \
                   .show(10) 

top-10 best year average rating


+-----------+------------------+---------+
|PublishYear|     AverageRating|BookCount|
+-----------+------------------+---------+
|       1983| 3.968961384820239|     1502|
|       1992|3.9631608884073657|     3692|
|       1986| 3.960931417979612|     2158|
|       1988| 3.960540768905788|     2367|
|       1989|3.9575137039937367|     2554|
|       1990|  3.95593340060545|     2973|
|       1991| 3.954548769371009|     3291|
|       1993|3.9543992796501155|     3887|
|       1994|3.9542364187456305|     4289|
|       1984|3.9533858267716537|     1905|
+-----------+------------------+---------+
only showing top 10 rows

top-10 worst year average rating
+-----------+------------------+---------+
|PublishYear|     AverageRating|BookCount|
+-----------+------------------+---------+
|       2008|3.9069505767321124|    13091|
|       2007|3.9145397965290276|    16710|
|       2009|3.9187002775208137|     2162|
|       2006|3.9253816189092805|    15146|
|       2004|3.9326649958228876|    11